## Import Libraries

In [75]:
from pyspark import SparkFiles
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, udf
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from graphframes import *
from pyspark.sql.types import FloatType

## Load Data

In [2]:
spark = SparkSession.builder.appName("GraphFlightsAnalysis") \
    .config("graphframes:graphframes:0.8.1-spark2.4-s_2.11") \
    .getOrCreate()

In [3]:
# Load datasets
train_path = "gs://msca-bdp-student-gcs/Group1/traindata.parquet/"

train_df = spark.read.parquet(train_path, header=True, inferSchema=True)

In [4]:
# Repartition the train data 
num = train_df.rdd.getNumPartitions()
print(num)
train_df = train_df.repartition(num)

20


In [4]:
# Show data
train_df.show(10)

+--------------------+----------+----------+---------------+------------------+--------------+-----------+------------+---------+--------------+----------------+--------+--------+--------------------+-----------------+------------+---------------+-------------+-----------------+-------------------+
|               legId|searchDate|flightDate|startingAirport|destinationAirport|travelDuration|elapsedDays|isRefundable|totalFare|seatsRemaining|DaysBeforeFlight|Layovers|NumStops|        AirlineNames|NumUniqueAirlines|AircraftType|NumUniqueCabins|hasFirstClass|FlightArrivalDate|totalTravelDistance|
+--------------------+----------+----------+---------------+------------------+--------------+-----------+------------+---------+--------------+----------------+--------+--------+--------------------+-----------------+------------+---------------+-------------+-----------------+-------------------+
|00000f65c6af0f881...|2022-05-25|2022-05-28|            DTW|               ATL|           185|      

## Creating the Graph Model

In [8]:
# Get unique airports
airports_id = train_df.select('startingAirport').distinct()

In [9]:
airports_id.show(airports_id.count())

+---------------+
|startingAirport|
+---------------+
|            OAK|
|            LGA|
|            BOS|
|            EWR|
|            DEN|
|            IAD|
|            CLT|
|            MIA|
|            DFW|
|            SFO|
|            ATL|
|            ORD|
|            DTW|
|            LAX|
|            JFK|
|            PHL|
+---------------+



In [10]:
# Get relevant columns from train df and drop duplicate flight itineraries 
flights = train_df.select("legID", "startingAirport", "destinationAirport",
                          "totalTravelDistance", "flightDate", "totalFare",
                          "travelDuration", "NumStops").dropDuplicates(["legID"])

In [25]:
flights.show(10)

+--------------------+---------------+------------------+-------------------+----------+---------+--------------+--------+
|               legID|startingAirport|destinationAirport|totalTravelDistance|flightDate|totalFare|travelDuration|NumStops|
+--------------------+---------------+------------------+-------------------+----------+---------+--------------+--------+
|00002f91c54a133e2...|            PHL|               ATL|               1467|2022-07-24|    636.6|           412|       1|
|0000562a70e90bddc...|            LGA|               SFO|               2968|2022-07-20|    432.6|           829|       2|
|0000c808a6ac95ca4...|            MIA|               BOS|               2054|2022-07-15|    247.6|           479|       1|
|00014bb3a18d6c00a...|            JFK|               ATL|                762|2022-09-20|     88.6|           147|       0|
|0001886895642ac68...|            DTW|               LGA|                808|2022-10-14|    371.6|           381|       1|
|0002d1d80a37529

In [11]:
# Create an id column for nodes
airports_id = airports_id.withColumn('id', airports_id['startingAirport'])

In [13]:
# Need to create a source and destination column for edges
flights = flights.withColumn("src", flights["startingAirport"])
flights = flights.withColumn("dst", flights["destinationAirport"])

In [17]:
# Create graph
graph = GraphFrame(airports_id, flights)

In [18]:
# Filter out itineraries with number of stops > 0 so that edges can represent nonstop flights 
filtered_edges = graph.edges.filter(col('NumStops') == 0)

In [19]:
# Create new graph using filtered edges 
graph = GraphFrame(graph.vertices, filtered_edges)

In [20]:
# Each airport is a vertex
graph.vertices.show(5)

+---------------+---+
|startingAirport| id|
+---------------+---+
|            OAK|OAK|
|            LGA|LGA|
|            BOS|BOS|
|            EWR|EWR|
|            DEN|DEN|
+---------------+---+
only showing top 5 rows



In [21]:
# Edges represent nonstop flights between two airports
graph.edges.show(5)

+--------------------+---------------+------------------+-------------------+----------+---------+--------------+--------+---+---+
|               legID|startingAirport|destinationAirport|totalTravelDistance|flightDate|totalFare|travelDuration|NumStops|src|dst|
+--------------------+---------------+------------------+-------------------+----------+---------+--------------+--------+---+---+
|0018c35572fd06c1a...|            ATL|               LAX|               1943|2022-06-04|    648.6|           291|       0|ATL|LAX|
|003bea4aa58c59ae1...|            SFO|               BOS|               2698|2022-10-02|    404.6|           340|       0|SFO|BOS|
|0040f5fc33d7edc6d...|            LAX|               PHL|               2397|2022-04-24|  1414.61|           310|       0|LAX|PHL|
|005781b3c6415a583...|            PHL|               BOS|                280|2022-11-13|    118.6|            81|       0|PHL|BOS|
|0092b2857f0864929...|            DFW|               CLT|                930|2022-0

In [22]:
# 16 unique airports in our dataset 
graph.vertices.count()

16

In [23]:
graph.edges.count()

644592

## Graph Analysis

#### Degree, Page Rank, Triangle Count, Clustering

In [24]:
# Number of incoming and outgoing flights
graph.degrees.orderBy("degree", ascending=False).show(10)

+---+------+
| id|degree|
+---+------+
|BOS|120403|
|ORD|117895|
|LAX|115600|
|LGA|104591|
|ATL|104264|
|JFK| 98487|
|DFW| 90792|
|EWR| 86256|
|SFO| 79922|
|MIA| 77125|
+---+------+
only showing top 10 rows



In [25]:
# Run PageRank until convergence 
results = graph.pageRank(resetProbability=0.15, tol= 0.01)
results.vertices.orderBy('pagerank',ascending=False).show()

+---------------+---+-------------------+
|startingAirport| id|           pagerank|
+---------------+---+-------------------+
|            LAX|LAX| 1.4186051569536233|
|            BOS|BOS| 1.4049228074329703|
|            ORD|ORD| 1.3773263737148511|
|            ATL|ATL|   1.25423441925382|
|            LGA|LGA|  1.235975294754727|
|            JFK|JFK| 1.1801389654814511|
|            DFW|DFW| 1.1012649067633775|
|            EWR|EWR| 1.0579451534658755|
|            SFO|SFO| 0.9886846083918923|
|            MIA|MIA| 0.9485271695016773|
|            CLT|CLT| 0.9354636554867383|
|            DEN|DEN| 0.8694803335579091|
|            DTW|DTW| 0.8114884045979907|
|            PHL|PHL| 0.6365554392629791|
|            IAD|IAD| 0.5786539644885018|
|            OAK|OAK|0.20073334689161776|
+---------------+---+-------------------+



In [26]:
# Run PageRank for a fixed number of iterations --> performs similar to the convergence version 
results = graph.pageRank(resetProbability=0.15, maxIter= 25)
results.vertices.orderBy('pagerank',ascending=False).show()

+---------------+---+------------------+
|startingAirport| id|          pagerank|
+---------------+---+------------------+
|            LAX|LAX|1.4352605011072015|
|            BOS|BOS|1.4095005729937669|
|            ORD|ORD|1.3832775659113188|
|            ATL|ATL|1.2585659665434674|
|            LGA|LGA|1.2362459065060005|
|            JFK|JFK|1.1790812344944301|
|            DFW|DFW|1.1020393493536234|
|            EWR|EWR| 1.058742971472584|
|            SFO|SFO|0.9874575535404675|
|            MIA|MIA| 0.946430694366115|
|            CLT|CLT|0.9341093611733323|
|            DEN|DEN|0.8686549113617154|
|            DTW|DTW|0.8089360395323112|
|            PHL|PHL|0.6315980590973331|
|            IAD|IAD|0.5713076392793238|
|            OAK|OAK| 0.188791673267007|
+---------------+---+------------------+



In [28]:
# Run PageRank for ORD --> ORD correctly comes up first 
results = graph.pageRank(resetProbability=0.15, maxIter= 25, sourceId = 'ORD')
results.vertices.orderBy('pagerank',ascending=False).show()

+---------------+---+--------------------+
|startingAirport| id|            pagerank|
+---------------+---+--------------------+
|            ORD|ORD|  0.2180725563198641|
|            BOS|BOS| 0.08331195668638448|
|            LGA|LGA| 0.08148322700193464|
|            LAX|LAX| 0.07394401230748432|
|            ATL|ATL| 0.06749774510159308|
|            JFK|JFK|  0.0611874026194233|
|            EWR|EWR| 0.05938438789650432|
|            DFW|DFW|0.059275652552730686|
|            SFO|SFO| 0.05116329160058725|
|            MIA|MIA| 0.04924754195885083|
|            CLT|CLT|0.048607483669572356|
|            DEN|DEN|0.044326096511362705|
|            DTW|DTW| 0.04287943290321142|
|            PHL|PHL|0.031222035185750226|
|            IAD|IAD|0.025971457138980174|
|            OAK|OAK|0.002425720545766...|
+---------------+---+--------------------+



In [29]:
# Count number of triangles for each airport
graph.triangleCount().orderBy("count", ascending = False).show(16)

+-----+---------------+---+
|count|startingAirport| id|
+-----+---------------+---+
|   92|            ATL|ATL|
|   92|            DTW|DTW|
|   92|            DEN|DEN|
|   92|            ORD|ORD|
|   92|            LAX|LAX|
|   86|            CLT|CLT|
|   86|            MIA|MIA|
|   86|            IAD|IAD|
|   86|            DFW|DFW|
|   86|            BOS|BOS|
|   82|            PHL|PHL|
|   76|            SFO|SFO|
|   72|            EWR|EWR|
|   55|            LGA|LGA|
|   55|            JFK|JFK|
|   21|            OAK|OAK|
+-----+---------------+---+



In [31]:
# Find which components each airport is in --> they're all in one connected compononent
sc = spark.sparkContext
sc.setCheckpointDir('graphframes_cps')

graph.connectedComponents().show(16)

24/12/03 20:57:36 WARN org.apache.spark.SparkContext: Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory 'graphframes_cps' appears to be on the local filesystem.


+---------------+---+------------+
|startingAirport| id|   component|
+---------------+---+------------+
|            OAK|OAK|111669149696|
|            LGA|LGA|111669149696|
|            BOS|BOS|111669149696|
|            EWR|EWR|111669149696|
|            DEN|DEN|111669149696|
|            IAD|IAD|111669149696|
|            CLT|CLT|111669149696|
|            MIA|MIA|111669149696|
|            DFW|DFW|111669149696|
|            SFO|SFO|111669149696|
|            ATL|ATL|111669149696|
|            ORD|ORD|111669149696|
|            DTW|DTW|111669149696|
|            LAX|LAX|111669149696|
|            JFK|JFK|111669149696|
|            PHL|PHL|111669149696|
+---------------+---+------------+



#### Queries 

In [32]:
# Average total fare of to all airports from ORD
graph.edges.filter("startingAirport = 'ORD'") \
    .groupBy("destinationAirport") \
    .agg(F.avg("totalFare").alias("averageTotalFare")) \
    .orderBy("averageTotalFare") \
    .show(15)

+------------------+------------------+
|destinationAirport|  averageTotalFare|
+------------------+------------------+
|               OAK|199.20094059405943|
|               LGA| 211.4218731251106|
|               JFK| 216.3852237851661|
|               DTW| 220.7035135900337|
|               ATL|229.58989970968562|
|               EWR|231.33618210306335|
|               PHL|236.13053206002706|
|               BOS| 245.5854786206888|
|               DFW|249.84711048951007|
|               DEN|252.52330939947743|
|               MIA|263.90394839718516|
|               IAD|276.21471408647164|
|               CLT|280.84893828067567|
|               LAX| 281.0057175588411|
|               SFO| 450.0419727488143|
+------------------+------------------+



In [46]:
# Average total distance to all airports from ORD
graph.edges.filter("startingAirport = 'ORD'") \
    .groupBy("destinationAirport") \
    .agg(F.avg("totalTravelDistance").alias("averageDistance")) \
    .orderBy("averageDistance") \
    .show(15)

+------------------+------------------+
|destinationAirport|   averageDistance|
+------------------+------------------+
|               DTW|243.12655719139298|
|               CLT| 595.8574577516532|
|               IAD| 598.8702928870293|
|               PHL| 676.0671896316508|
|               ATL| 703.8844022169438|
|               LGA| 723.2874536791953|
|               JFK| 723.7973145780052|
|               EWR| 723.9940807799443|
|               DFW| 861.7826573426573|
|               BOS| 865.1710344827586|
|               DEN| 904.6566579634465|
|               MIA|1237.6899921813917|
|               OAK|            1467.0|
|               LAX|1696.4759425119767|
|               SFO|1846.2120853080569|
+------------------+------------------+



In [47]:
# Average travel duration to all airports from ORD
graph.edges.filter("startingAirport = 'ORD'") \
    .groupBy("destinationAirport") \
    .agg(F.avg("travelDuration").alias("averageDuration")) \
    .orderBy("averageDuration") \
    .show(15)

+------------------+------------------+
|destinationAirport|   averageDuration|
+------------------+------------------+
|               DTW| 83.75339750849378|
|               IAD| 115.2133891213389|
|               ATL|120.60095011876484|
|               CLT|120.73108008817046|
|               PHL| 122.6606412005457|
|               EWR|130.31354456824513|
|               LGA|131.36050820539967|
|               BOS| 142.7582068965517|
|               JFK|142.82608695652175|
|               DFW| 149.3795804195804|
|               DEN|162.50685378590077|
|               MIA| 186.7212666145426|
|               LAX|265.53842949385546|
|               OAK| 278.3811881188119|
|               SFO|280.61167061611377|
+------------------+------------------+



In [33]:
# Longest flight routes from each airport by total travel distance
graph.edges.groupBy("src", "dst") \
    .max("totalTravelDistance") \
    .orderBy("max(totalTravelDistance)", ascending = False) \
    .show(15)

+---+---+------------------------+
|src|dst|max(totalTravelDistance)|
+---+---+------------------------+
|SFO|BOS|                    2698|
|BOS|SFO|                    2698|
|LAX|BOS|                    2606|
|BOS|LAX|                    2606|
|LAX|PHL|                    2606|
|MIA|SFO|                    2582|
|SFO|MIA|                    2582|
|SFO|JFK|                    2577|
|JFK|SFO|                    2577|
|EWR|SFO|                    2572|
|SFO|EWR|                    2572|
|SFO|DTW|                    2566|
|SFO|PHL|                    2516|
|PHL|SFO|                    2516|
|JFK|LAX|                    2467|
+---+---+------------------------+
only showing top 15 rows



In [37]:
# Shortest flight routes from each airport by total travel distance
graph.edges.groupBy("src", "dst") \
    .min("totalTravelDistance") \
    .orderBy("min(totalTravelDistance)", ascending = True) \
    .show(15)

+---+---+------------------------+
|src|dst|min(totalTravelDistance)|
+---+---+------------------------+
|EWR|PHL|                      89|
|PHL|EWR|                      89|
|EWR|BOS|                     185|
|JFK|BOS|                     185|
|BOS|EWR|                     185|
|EWR|LGA|                     185|
|BOS|JFK|                     185|
|LGA|PHL|                     185|
|BOS|LGA|                     185|
|LGA|BOS|                     185|
|JFK|IAD|                     221|
|IAD|JFK|                     221|
|EWR|IAD|                     221|
|LGA|IAD|                     221|
|IAD|EWR|                     221|
+---+---+------------------------+
only showing top 15 rows



In [36]:
# Longest flight routes from ORD by total travel distance
graph.edges.filter("startingAirport='ORD'") \
    .groupBy("src", "dst") \
    .max("totalTravelDistance") \
    .orderBy("max(totalTravelDistance)", ascending = False) \
    .show()

+---+---+------------------------+
|src|dst|max(totalTravelDistance)|
+---+---+------------------------+
|ORD|SFO|                    1847|
|ORD|LAX|                    1745|
|ORD|OAK|                    1467|
|ORD|BOS|                    1467|
|ORD|DTW|                    1467|
|ORD|CLT|                    1467|
|ORD|DEN|                    1467|
|ORD|PHL|                    1467|
|ORD|IAD|                    1467|
|ORD|MIA|                    1467|
|ORD|ATL|                    1467|
|ORD|DFW|                    1467|
|ORD|JFK|                    1467|
|ORD|LGA|                    1467|
|ORD|EWR|                    1467|
+---+---+------------------------+



In [38]:
# Shortest flight routes from ORD by total travel distance
graph.edges.filter("startingAirport='ORD'") \
    .groupBy("src", "dst") \
    .min("totalTravelDistance") \
    .orderBy("min(totalTravelDistance)") \
    .show()

+---+---+------------------------+
|src|dst|min(totalTravelDistance)|
+---+---+------------------------+
|ORD|DTW|                     240|
|ORD|CLT|                     592|
|ORD|IAD|                     594|
|ORD|ATL|                     600|
|ORD|PHL|                     672|
|ORD|JFK|                     720|
|ORD|LGA|                     720|
|ORD|EWR|                     720|
|ORD|DFW|                     799|
|ORD|BOS|                     862|
|ORD|DEN|                     903|
|ORD|MIA|                    1192|
|ORD|OAK|                    1467|
|ORD|SFO|                    1467|
|ORD|LAX|                    1467|
+---+---+------------------------+



In [50]:
# Longest flight routes from ORD by total travel time
graph.edges.filter("startingAirport='ORD'") \
    .groupBy("src", "dst") \
    .max("travelDuration") \
    .orderBy("max(travelDuration)", ascending = False) \
    .show()

+---+---+-------------------+
|src|dst|max(travelDuration)|
+---+---+-------------------+
|ORD|SFO|                296|
|ORD|OAK|                288|
|ORD|LAX|                284|
|ORD|MIA|                203|
|ORD|DEN|                177|
|ORD|DFW|                171|
|ORD|BOS|                158|
|ORD|JFK|                158|
|ORD|LGA|                154|
|ORD|ATL|                146|
|ORD|EWR|                146|
|ORD|PHL|                141|
|ORD|CLT|                138|
|ORD|IAD|                125|
|ORD|DTW|                 92|
+---+---+-------------------+



In [48]:
# Shortest flight routes from ORD by total travel time
graph.edges.filter("startingAirport='ORD'") \
    .groupBy("src", "dst") \
    .min("travelDuration") \
    .orderBy("min(travelDuration)") \
    .show()

+---+---+-------------------+
|src|dst|min(travelDuration)|
+---+---+-------------------+
|ORD|DTW|                 75|
|ORD|IAD|                106|
|ORD|ATL|                109|
|ORD|PHL|                112|
|ORD|CLT|                112|
|ORD|EWR|                119|
|ORD|LGA|                119|
|ORD|JFK|                129|
|ORD|BOS|                132|
|ORD|DFW|                137|
|ORD|DEN|                149|
|ORD|MIA|                175|
|ORD|LAX|                242|
|ORD|SFO|                260|
|ORD|OAK|                275|
+---+---+-------------------+



In [39]:
# Routes with most nonstop flights 
# Storing the result for later use 
flightCount = graph.edges.groupBy("src", "dst").count().orderBy("count", ascending = False)

In [41]:
flightCount.show()

+---+---+-----+
|src|dst|count|
+---+---+-----+
|ORD|LGA|11334|
|LGA|ORD|11152|
|JFK|LAX|10005|
|LAX|JFK| 9857|
|LGA|BOS| 8929|
|BOS|LGA| 8762|
|LAX|SFO| 8553|
|SFO|LAX| 8248|
|BOS|JFK| 8200|
|JFK|BOS| 7996|
|BOS|ORD| 7286|
|SFO|JFK| 7257|
|ORD|BOS| 7250|
|JFK|SFO| 7116|
|LGA|DFW| 6763|
|DFW|LGA| 6693|
|MIA|JFK| 6615|
|JFK|MIA| 6502|
|ORD|EWR| 5744|
|MIA|LGA| 5706|
+---+---+-----+
only showing top 20 rows



In [42]:
# Routes with least amount of nonstop flights
graph.edges.groupBy("src", "dst").count().orderBy("count").show(10)

+---+---+-----+
|src|dst|count|
+---+---+-----+
|LGA|PHL|    3|
|LGA|LAX|    7|
|LAX|LGA|    8|
|OAK|DTW|   21|
|DTW|OAK|   22|
|OAK|EWR|  122|
|EWR|OAK|  140|
|OAK|ORD|  140|
|DEN|OAK|  157|
|OAK|DEN|  159|
+---+---+-----+
only showing top 10 rows



In [44]:
# Routes from ORD with most amount of flights
graph.edges.filter("src = 'ORD'").groupBy("src", "dst").count().orderBy("count", ascending = False).show(15)

+---+---+-----+
|src|dst|count|
+---+---+-----+
|ORD|LGA|11334|
|ORD|BOS| 7250|
|ORD|EWR| 5744|
|ORD|LAX| 4800|
|ORD|ATL| 3789|
|ORD|DFW| 3575|
|ORD|DTW| 3532|
|ORD|SFO| 3376|
|ORD|JFK| 3128|
|ORD|DEN| 3064|
|ORD|PHL| 2932|
|ORD|CLT| 2722|
|ORD|MIA| 2558|
|ORD|IAD| 1434|
|ORD|OAK|  202|
+---+---+-----+



In [45]:
# Routes from ORD with least amount of flights
graph.edges.filter("src = 'ORD'").groupBy("src", "dst").count().orderBy("count").show(15)

+---+---+-----+
|src|dst|count|
+---+---+-----+
|ORD|OAK|  202|
|ORD|IAD| 1434|
|ORD|MIA| 2558|
|ORD|CLT| 2722|
|ORD|PHL| 2932|
|ORD|DEN| 3064|
|ORD|JFK| 3128|
|ORD|SFO| 3376|
|ORD|DTW| 3532|
|ORD|DFW| 3575|
|ORD|ATL| 3789|
|ORD|LAX| 4801|
|ORD|EWR| 5744|
|ORD|BOS| 7250|
|ORD|LGA|11334|
+---+---+-----+



#### Shortest Paths and Motif Finding

In [51]:
# Shortest Paths from ORD --> nonstop flight to every airport
paths = graph.shortestPaths(landmarks=["ORD"])
paths.show()

+---------------+---+----------+
|startingAirport| id| distances|
+---------------+---+----------+
|            LAX|LAX|{ORD -> 1}|
|            BOS|BOS|{ORD -> 1}|
|            MIA|MIA|{ORD -> 1}|
|            DEN|DEN|{ORD -> 1}|
|            LGA|LGA|{ORD -> 1}|
|            OAK|OAK|{ORD -> 1}|
|            DTW|DTW|{ORD -> 1}|
|            IAD|IAD|{ORD -> 1}|
|            DFW|DFW|{ORD -> 1}|
|            EWR|EWR|{ORD -> 1}|
|            ORD|ORD|{ORD -> 0}|
|            ATL|ATL|{ORD -> 1}|
|            JFK|JFK|{ORD -> 1}|
|            SFO|SFO|{ORD -> 1}|
|            CLT|CLT|{ORD -> 1}|
|            PHL|PHL|{ORD -> 1}|
+---------------+---+----------+



In [52]:
# Shortest Paths from JFK and EWR
# EWR has nonstop flight to each airport except for JFK and LGA
# JFK has nonstop flight to each airport except for LGA, OAK, EWR, and PHL

paths = graph.shortestPaths(landmarks=["JFK", "EWR"])
paths.show()

+---------------+---+--------------------+
|startingAirport| id|           distances|
+---------------+---+--------------------+
|            LAX|LAX|{EWR -> 1, JFK -> 1}|
|            BOS|BOS|{EWR -> 1, JFK -> 1}|
|            MIA|MIA|{EWR -> 1, JFK -> 1}|
|            DEN|DEN|{EWR -> 1, JFK -> 1}|
|            LGA|LGA|{EWR -> 2, JFK -> 2}|
|            OAK|OAK|{EWR -> 1, JFK -> 2}|
|            DTW|DTW|{EWR -> 1, JFK -> 1}|
|            IAD|IAD|{EWR -> 1, JFK -> 1}|
|            DFW|DFW|{EWR -> 1, JFK -> 1}|
|            EWR|EWR|{EWR -> 0, JFK -> 2}|
|            ORD|ORD|{EWR -> 1, JFK -> 1}|
|            ATL|ATL|{EWR -> 1, JFK -> 1}|
|            JFK|JFK|{EWR -> 2, JFK -> 0}|
|            SFO|SFO|{EWR -> 1, JFK -> 1}|
|            CLT|CLT|{EWR -> 1, JFK -> 1}|
|            PHL|PHL|{EWR -> 1, JFK -> 2}|
+---------------+---+--------------------+



In [31]:
flightCount = graph.edges.filter("NumStops == 0").groupBy("src", "dst").count().orderBy("count", ascending = False)

In [53]:
# Flight routes btwn a and c with no direct connection 
subGraph = GraphFrame(graph.vertices.select("id"), flightCount)
res = subGraph.find("(a)-[]->(b); (b)-[]->(c); !(a)-[]->(c)").filter("c.id !=a.id")

In [54]:
res.show()

+-----+-----+-----+
|    a|    b|    c|
+-----+-----+-----+
|{OAK}|{DEN}|{SFO}|
|{LGA}|{DEN}|{JFK}|
|{OAK}|{DFW}|{ORD}|
|{SFO}|{ATL}|{LGA}|
|{JFK}|{IAD}|{PHL}|
|{OAK}|{DEN}|{MIA}|
|{SFO}|{ORD}|{LGA}|
|{SFO}|{IAD}|{PHL}|
|{OAK}|{PHL}|{CLT}|
|{DEN}|{IAD}|{PHL}|
|{OAK}|{DFW}|{BOS}|
|{PHL}|{SFO}|{JFK}|
|{LGA}|{ATL}|{PHL}|
|{OAK}|{LAX}|{SFO}|
|{OAK}|{PHL}|{DFW}|
|{ATL}|{DTW}|{OAK}|
|{LGA}|{LAX}|{JFK}|
|{JFK}|{LAX}|{LGA}|
|{MIA}|{IAD}|{PHL}|
|{DFW}|{PHL}|{OAK}|
+-----+-----+-----+
only showing top 20 rows



In [55]:
# Finding chain of 3 flights between airports 
subGraph = GraphFrame(graph.vertices.select("id"), flightCount)
res = subGraph.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(d)")

In [56]:
res.show()

+-----+-----+-----+-----+
|    a|    b|    c|    d|
+-----+-----+-----+-----+
|{ORD}|{OAK}|{DTW}|{LGA}|
|{ORD}|{OAK}|{DTW}|{DFW}|
|{ORD}|{OAK}|{DTW}|{LAX}|
|{ORD}|{OAK}|{DTW}|{BOS}|
|{ORD}|{OAK}|{DTW}|{JFK}|
|{ORD}|{OAK}|{DTW}|{ATL}|
|{ORD}|{OAK}|{DTW}|{IAD}|
|{ORD}|{OAK}|{DTW}|{CLT}|
|{ORD}|{OAK}|{DTW}|{SFO}|
|{ORD}|{OAK}|{DTW}|{ORD}|
|{ORD}|{OAK}|{DTW}|{EWR}|
|{ORD}|{OAK}|{DTW}|{MIA}|
|{ORD}|{OAK}|{DTW}|{OAK}|
|{ORD}|{OAK}|{DTW}|{DEN}|
|{ORD}|{OAK}|{DTW}|{PHL}|
|{ORD}|{OAK}|{ORD}|{LAX}|
|{ORD}|{OAK}|{ORD}|{SFO}|
|{ORD}|{OAK}|{ORD}|{EWR}|
|{ORD}|{OAK}|{ORD}|{JFK}|
|{ORD}|{OAK}|{ORD}|{LGA}|
+-----+-----+-----+-----+
only showing top 20 rows



In [58]:
# Chain of 3 flights starting from an airport other than ORD
res.filter("a.id != 'ORD'").show()

+-----+-----+-----+-----+
|    a|    b|    c|    d|
+-----+-----+-----+-----+
|{DTW}|{OAK}|{DTW}|{LGA}|
|{DTW}|{OAK}|{DTW}|{DFW}|
|{DTW}|{OAK}|{DTW}|{LAX}|
|{DTW}|{OAK}|{DTW}|{BOS}|
|{DTW}|{OAK}|{DTW}|{JFK}|
|{DTW}|{OAK}|{DTW}|{ATL}|
|{DTW}|{OAK}|{DTW}|{IAD}|
|{DTW}|{OAK}|{DTW}|{CLT}|
|{DTW}|{OAK}|{DTW}|{SFO}|
|{DTW}|{OAK}|{DTW}|{ORD}|
|{DTW}|{OAK}|{DTW}|{EWR}|
|{DTW}|{OAK}|{DTW}|{MIA}|
|{DTW}|{OAK}|{DTW}|{OAK}|
|{DTW}|{OAK}|{DTW}|{DEN}|
|{DTW}|{OAK}|{DTW}|{PHL}|
|{DTW}|{OAK}|{ORD}|{LAX}|
|{DTW}|{OAK}|{ORD}|{SFO}|
|{DTW}|{OAK}|{ORD}|{EWR}|
|{DTW}|{OAK}|{ORD}|{JFK}|
|{DTW}|{OAK}|{ORD}|{LGA}|
+-----+-----+-----+-----+
only showing top 20 rows



In [64]:
# Find multi-city trips through ORD as start and stop 
subGraph = GraphFrame(graph.vertices.select("id"), flightCount)
res = subGraph.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(d); (d)-[]->(e)")
res.filter("a.id == 'ORD' AND e.id == 'ORD'").show()

+-----+-----+-----+-----+-----+
|    a|    b|    c|    d|    e|
+-----+-----+-----+-----+-----+
|{ORD}|{IAD}|{OAK}|{DTW}|{ORD}|
|{ORD}|{IAD}|{OAK}|{LAX}|{ORD}|
|{ORD}|{IAD}|{OAK}|{DEN}|{ORD}|
|{ORD}|{IAD}|{OAK}|{EWR}|{ORD}|
|{ORD}|{IAD}|{OAK}|{DFW}|{ORD}|
|{ORD}|{IAD}|{OAK}|{PHL}|{ORD}|
|{ORD}|{DEN}|{OAK}|{DTW}|{ORD}|
|{ORD}|{DEN}|{OAK}|{LAX}|{ORD}|
|{ORD}|{DEN}|{OAK}|{DEN}|{ORD}|
|{ORD}|{DEN}|{OAK}|{EWR}|{ORD}|
|{ORD}|{DEN}|{OAK}|{DFW}|{ORD}|
|{ORD}|{DEN}|{OAK}|{PHL}|{ORD}|
|{ORD}|{DTW}|{OAK}|{DTW}|{ORD}|
|{ORD}|{DTW}|{OAK}|{LAX}|{ORD}|
|{ORD}|{DTW}|{OAK}|{DEN}|{ORD}|
|{ORD}|{DTW}|{OAK}|{EWR}|{ORD}|
|{ORD}|{DTW}|{OAK}|{DFW}|{ORD}|
|{ORD}|{DTW}|{OAK}|{PHL}|{ORD}|
|{ORD}|{LAX}|{OAK}|{DTW}|{ORD}|
|{ORD}|{LAX}|{OAK}|{LAX}|{ORD}|
+-----+-----+-----+-----+-----+
only showing top 20 rows



In [65]:
# Multi-city trips continued but seeing results without the 3rd airport as Oakland for more recommendations 
res.filter("a.id == 'ORD' AND c.id != 'OAK' AND e.id == 'ORD'").show()

+-----+-----+-----+-----+-----+
|    a|    b|    c|    d|    e|
+-----+-----+-----+-----+-----+
|{ORD}|{BOS}|{DEN}|{OAK}|{ORD}|
|{ORD}|{ATL}|{ORD}|{OAK}|{ORD}|
|{ORD}|{DFW}|{ORD}|{OAK}|{ORD}|
|{ORD}|{DFW}|{DEN}|{OAK}|{ORD}|
|{ORD}|{DTW}|{DEN}|{OAK}|{ORD}|
|{ORD}|{CLT}|{DTW}|{OAK}|{ORD}|
|{ORD}|{IAD}|{LAX}|{OAK}|{ORD}|
|{ORD}|{MIA}|{ORD}|{OAK}|{ORD}|
|{ORD}|{EWR}|{DTW}|{OAK}|{ORD}|
|{ORD}|{SFO}|{DEN}|{OAK}|{ORD}|
|{ORD}|{CLT}|{ORD}|{OAK}|{ORD}|
|{ORD}|{PHL}|{EWR}|{OAK}|{ORD}|
|{ORD}|{IAD}|{EWR}|{OAK}|{ORD}|
|{ORD}|{ATL}|{PHL}|{OAK}|{ORD}|
|{ORD}|{BOS}|{DTW}|{OAK}|{ORD}|
|{ORD}|{MIA}|{LAX}|{OAK}|{ORD}|
|{ORD}|{LGA}|{DTW}|{OAK}|{ORD}|
|{ORD}|{BOS}|{EWR}|{OAK}|{ORD}|
|{ORD}|{DEN}|{PHL}|{OAK}|{ORD}|
|{ORD}|{PHL}|{ORD}|{OAK}|{ORD}|
+-----+-----+-----+-----+-----+
only showing top 20 rows



In [66]:
# Finding circular routes
subGraph = GraphFrame(graph.vertices.select("id"), flightCount)
res = subGraph.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)")
res.show()

+-----+-----+-----+
|    a|    b|    c|
+-----+-----+-----+
|{PHL}|{OAK}|{DTW}|
|{PHL}|{OAK}|{ORD}|
|{PHL}|{OAK}|{LAX}|
|{PHL}|{OAK}|{DEN}|
|{PHL}|{OAK}|{EWR}|
|{DEN}|{OAK}|{DTW}|
|{DEN}|{OAK}|{ORD}|
|{DEN}|{OAK}|{LAX}|
|{DEN}|{OAK}|{EWR}|
|{DEN}|{OAK}|{PHL}|
|{ORD}|{OAK}|{DTW}|
|{ORD}|{OAK}|{LAX}|
|{ORD}|{OAK}|{DEN}|
|{ORD}|{OAK}|{EWR}|
|{ORD}|{OAK}|{PHL}|
|{EWR}|{OAK}|{DTW}|
|{EWR}|{OAK}|{ORD}|
|{EWR}|{OAK}|{LAX}|
|{EWR}|{OAK}|{DEN}|
|{EWR}|{OAK}|{PHL}|
+-----+-----+-----+
only showing top 20 rows



In [71]:
# Finding airports with flights to the same airport
subGraph = GraphFrame(graph.vertices.select("id"), flightCount)
res = subGraph.find("(a)-[]->(b); (c)-[]->(b)").filter("a.id != c.id")
res.show()

+-----+-----+-----+
|    a|    b|    c|
+-----+-----+-----+
|{PHL}|{OAK}|{EWR}|
|{PHL}|{OAK}|{DEN}|
|{PHL}|{OAK}|{IAD}|
|{PHL}|{OAK}|{LAX}|
|{PHL}|{OAK}|{DTW}|
|{PHL}|{OAK}|{ORD}|
|{EWR}|{OAK}|{DEN}|
|{EWR}|{OAK}|{IAD}|
|{EWR}|{OAK}|{PHL}|
|{EWR}|{OAK}|{LAX}|
|{EWR}|{OAK}|{DTW}|
|{EWR}|{OAK}|{ORD}|
|{ORD}|{OAK}|{EWR}|
|{ORD}|{OAK}|{DEN}|
|{ORD}|{OAK}|{IAD}|
|{ORD}|{OAK}|{PHL}|
|{ORD}|{OAK}|{LAX}|
|{ORD}|{OAK}|{DTW}|
|{DEN}|{OAK}|{EWR}|
|{DEN}|{OAK}|{IAD}|
+-----+-----+-----+
only showing top 20 rows



## Link Prediction Model

In [74]:
# Conduct link prediction using Jaccard Index

# Count neighbors for each airport
neighbors_a = graph.edges.groupBy('src').agg(F.collect_list('dst').alias('a_neighbors'))
neighbors_b = graph.edges.groupBy('dst').agg(F.collect_list('src').alias('b_neighbors'))

# Join the neighbors data to get pairs (a, b) that are not directly connected
neighbor_pairs = neighbors_a.crossJoin(neighbors_b) \
    .filter('src != dst')

# Calculate Jaccard Index for each input airport pair 
def jaccard_index(a_neighbors, b_neighbors):
    # get intersection of neighbors sets 
    intersection = set(a_neighbors).intersection(b_neighbors)
    # join both neihbors sets
    union = set(a_neighbors).union(b_neighbors)
    # calculate the inde
    return len(intersection) / len(union) if len(union) > 0 else 0

In [76]:
# Create the UDF 
jaccard_udf = udf(jaccard_index, FloatType())

# Calculate Jaccard Index to each pair of airports
predictions = neighbor_pairs.withColumn('jaccard_index', jaccard_udf('a_neighbors', 'b_neighbors'))

# Show prediction results 
predictions.show()

+---+--------------------+---+--------------------+-------------+
|src|         a_neighbors|dst|         b_neighbors|jaccard_index|
+---+--------------------+---+--------------------+-------------+
|LGA|[BOS, CLT, ATL, A...|OAK|[DEN, EWR, LAX, P...|   0.41666666|
|BOS|[JFK, LGA, DEN, D...|OAK|[DEN, EWR, LAX, P...|          0.5|
|EWR|[ORD, DFW, CLT, C...|OAK|[DEN, EWR, LAX, P...|   0.42857143|
|DEN|[LAX, ATL, EWR, L...|OAK|[DEN, EWR, LAX, P...|        0.375|
|IAD|[DEN, DTW, LAX, C...|OAK|[DEN, EWR, LAX, P...|   0.33333334|
|CLT|[DEN, ATL, IAD, I...|OAK|[DEN, EWR, LAX, P...|          0.5|
|MIA|[EWR, PHL, PHL, J...|OAK|[DEN, EWR, LAX, P...|          0.5|
|DFW|[LAX, DTW, JFK, L...|OAK|[DEN, EWR, LAX, P...|          0.5|
|SFO|[EWR, DTW, BOS, B...|OAK|[DEN, EWR, LAX, P...|   0.53846157|
|ATL|[CLT, LAX, LAX, L...|OAK|[DEN, EWR, LAX, P...|          0.5|
|ORD|[EWR, LGA, LGA, P...|OAK|[DEN, EWR, LAX, P...|        0.375|
|DTW|[BOS, LGA, ORD, C...|OAK|[DEN, EWR, LAX, P...|        0.375|
|LAX|[BOS,

In [77]:
# Filter prediction results for ORD 
predictions.filter("src == 'ORD'").show()

+---+--------------------+---+--------------------+-------------+
|src|         a_neighbors|dst|         b_neighbors|jaccard_index|
+---+--------------------+---+--------------------+-------------+
|ORD|[SFO, DFW, SFO, P...|OAK|[LAX, LAX, ORD, E...|       0.3125|
|ORD|[SFO, DFW, SFO, P...|LGA|[ORD, DTW, IAD, M...|        0.625|
|ORD|[SFO, DFW, SFO, P...|BOS|[LGA, EWR, ORD, L...|       0.8125|
|ORD|[SFO, DFW, SFO, P...|EWR|[MIA, BOS, ORD, O...|         0.75|
|ORD|[SFO, DFW, SFO, P...|DEN|[LAX, IAD, LAX, S...|        0.875|
|ORD|[SFO, DFW, SFO, P...|IAD|[LGA, BOS, CLT, D...|         0.75|
|ORD|[SFO, DFW, SFO, P...|CLT|[PHL, PHL, JFK, L...|       0.8125|
|ORD|[SFO, DFW, SFO, P...|MIA|[ORD, DFW, DTW, C...|       0.8125|
|ORD|[SFO, DFW, SFO, P...|DFW|[EWR, LAX, ATL, E...|        0.875|
|ORD|[SFO, DFW, SFO, P...|SFO|[LAX, BOS, EWR, M...|         0.75|
|ORD|[SFO, DFW, SFO, P...|ATL|[ORD, IAD, LAX, J...|       0.8125|
|ORD|[SFO, DFW, SFO, P...|DTW|[ORD, CLT, BOS, M...|        0.875|
|ORD|[SFO,